In [ ]:
# import required packages 
using DataFrames

In [ ]:
# Function for Confusion Matrix
function confusion_matrix(y_true::Array{Int64,1},y_pred::Array{Int64,1})
    # Generate confusion matrix
    classes = sort(unique([unique(y_true),unique(y_pred)]))
    cm = zeros(Int64,length(classes),length(classes))

    for i in 1:length(y_test)
        # translate label to index
        true_class = findfirst(classes,y_test[i])
        pred_class = findfirst(classes,y_pred[i])
        # pred class is the row, true class is the column
        cm[pred_class,true_class] += 1
    end
    cm
end

# Example: confusion_matrix(y_test,y_pred)

In [ ]:
# Partition Data into Train and Test datasets
function partitionTrainTest(data, at = 0.7)
    n = nrow(data)
    idx = shuffle(1:n)
    train_idx = view(idx, 1:floor(Int, at*n))
    test_idx = view(idx, (floor(Int, at*n)+1):n)
    data[train_idx,:], data[test_idx,:]
end

# Example: train,test = partitionTrainTest(data, 0.7) # 70% train

In [ ]:
# Build a standard scaler function
type StandardScalar
    mean::Vector{Float64}
    std::Vector{Float64}
end

# Helper function to initialize an empty scalar
function StandardScalar()
    StandardScalar(Array(Real,0),Array(Real,0))
end

# Compute mean and standard deviation of each column
function fit_std_scalar!(std_scalar::StandardScalar,X::Matrix{Real})
    n_rows, n_cols = size(X_test)
    std_scalar.std = zeros(n_cols)
    std_scalar.mean = zeros(n_cols)
    # for loops are fast again!
    for i = 1:n_cols
        std_scalar.mean[i] = mean(X[:,i])
        std_scalar.std[i] = std(X[:,i])
    end
end

function transform(std_scalar::StandardScalar,X::Matrix{Real})
    (X .- std_scalar.mean') ./ std_scalar.std' # broadcasting fu
end

# fit and transform in one function
function fit_transform!(std_scalar::StandardScalar,X::Matrix{Real})
    fit_std_scalar!(std_scalar,X)
    transform(std_scalar,X)
end

# Examples:
# Perform Standard Scaling for all X variables
# std_scalar = StandardScalar()
# X_train = fit_transform!(std_scalar,X_train)
# X_test = transform(std_scalar,X_test)

In [ ]:
# Logistic Regression
function predict(data, model_params) 
	1.0 ./ (1.0 + exp(-data * model_params[:omega]  - model_params[:beta]))
end 

function _mult(a::Array{Float64,1},b::Array{Float64,2})
    result = zeros(length(a))
    both_non_zero_indicator = ((a .!= 0) &amp; (b .!= 0))
    result[both_non_zero_indicator[:]] = a[both_non_zero_indicator[:]] .* b[both_non_zero_indicator]
    return result
end


function goal_function(omega::Array{Float64,2}, beta::Float64, data::Array{Float64,2}, labels::Array{Float64,1})
    f_partial = 1.0 ./ (1.0 + exp(-data * omega  - beta))
    result = -sum(_mult(labels, log(f_partial)) + _mult((1.0 - labels), log(1.0 - f_partial)))
    return result
end

function convergence(omega::Array{Float64,2}, beta::Float64, data::Array{Float64,2}, labels::Array{Float64,1},  prevJ::Float64, epsilon::Float64)
     currJ = goal_function(omega, beta, data, labels)
     return abs(prevJ - currJ) &lt; epsilon
end

function update_params(omega::Array{Float64,2}, beta::Float64,data::Array{Float64,2}, labels::Array{Float64,1}, alpha::Float64)
    partial_derivative = (1.0 ./ (1.0 + exp(-data * omega  - beta)))  - labels
    omega = omega - alpha *  (partial_derivative' * data)'
    beta = beta  - alpha * sum(partial_derivative)
    return omega,beta
end

function logistic_regression(data::Array{Float64,2}, labels::Array{Float64,1}, params::Dict{Symbol,Float64})

    omega = zeros(Float64, size(data,2),1)
    beta = 0.0
    J = Inf
    current_iter = 0
    alpha_step, epsilon, max_iter = params[:alpha], params[:eps], params[:max_iter]

    while !convergence(omega, beta, data, labels, J, epsilon) &amp;&amp; current_iter &lt; max_iter
         J = goal_function(omega, beta, data, labels)
         omega, beta = update_params(omega, beta, data, labels, alpha_step)
         current_iter += 1
    end
    model_params = Dict();
    model_params[:omega] = omega; model_params[:beta] = beta;	
    return model_params
end

In [ ]:
NeuralNetwork = function(params)
    instance = x -> f(x,params) #  Instance of a neural network is the set of function parameters [params] + some constant behaviour [f].
    return instance 
end

function appendColumnOfOnes(a::Array{Float64,2})
  vcat(a,ones(1,size(a ,2)))
end

function sigmoidNeuronTransformFunction(params, input)
  return 1.0 ./ (1.0 .+ exp(-params * appendColumnOfOnes(input)))
end

abstract Layer

type FullyConnectedComputingLayer <: Layer
  inputSize::Int64
  numberOfNeurons::Int64
  parameters::Array{Float64,2}
  transform::Function # we don't define the function directly to get flexibility later 

  function FullyConnectedComputingLayer(inputSize, numberOfNeurons, transform::Function)
    parameters = randn(numberOfNeurons, inputSize + 1) # adding one param column for bias
    return new(inputSize, numberOfNeurons, parameters, transform)
  end
end

type NetworkArchitecture
  layers::Array{Layer}
  function NetworkArchitecture(firstLayer::Layer)
    return new([firstLayer])
  end
end

function addFullyConnectedSigmoidLayer(architecture::NetworkArchitecture, numberOfNeurons::Int64)
 lastNetworkLayer = architecture.layers[end]
 inputSize = lastNetworkLayer.numberOfNeurons
 sigmoidLayer = FullyConnectedComputingLayer(inputSize, numberOfNeurons, sigmoidNeuronTransformFunction)
 push!(architecture.layers, sigmoidLayer)
end

sigmoidLayer = FullyConnectedComputingLayer(15, 32, sigmoidNeuronTransformFunction) 
architecture = NetworkArchitecture(sigmoidLayer)
size(architecture.layers[end].parameters)
# (15,32)

addFullyConnectedSigmoidLayer(architecture, 16)
size(architecture.layers[end].parameters)
# (16,32)

function infer(architecture::NetworkArchitecture, input)
  currentResult = input
  for i in 1:length(architecture.layers)
     layer = architecture.layers[i]
     currentResult = layer.transform(layer.parameters, currentResult)
  end
  return currentResult
end

firstLayer = FullyConnectedComputingLayer(14, 2031, sigmoidNeuronTransformFunction)
architecture = NetworkArchitecture(firstLayer)
addFullyConnectedSigmoidLayer(architecture, 16)
addFullyConnectedSigmoidLayer(architecture, 8)
addFullyConnectedSigmoidLayer(architecture, 1)
infer(architecture , features) # we try a single vector now
# 5x1 Array{Float64,2}:
# 0.217054
# 0.681068
# 0.149886
# 0.772127
# 0.458145

In [ ]:
function sigmoid(z)
  # sigmoid is a basic sigmoid function returning values from 0-1
  1. / ( 1. + 1e.^(-z) )
end

In [ ]:
function sigmoidGradient(z)
  sigmoid(z) .* ( 1 - sigmoid(z) )
end

In [ ]:
function initialize_theta(input_unit_count, output_class_count, hidden_unit_length_list)
  #
  #initialize_theta creates architecture of neural network
  #    
  #Parameters:
  #  hidden_unit_length_list - Array of hidden layer units
  #  input_unit_count - integer, number of input units (features)
  #  output_class_count - integer, number of output classes
  #
  #Returns:
  #  
  #Array of theta arrays randomly initialized to from -.5 to .5
  #
  
  if length( hidden_unit_length_list ) == 0
    hidden_unit_length_list = [2]
  end
  
  # unit_count_list = [input_unit_count]
  # unit_count_list = [unit_count_list, hidden_unit_length_list]
  # unit_count_list = [unit_count_list, output_class_count]
  unit_count_list = [input_unit_count, output_class_count]
  layers = length(unit_count_list)
  
  Theta_L = [rand(unit_count_list[i], unit_count_list[i-1]+1) - 0.5 for i = 2:layers]

end

In [ ]:
function print_theta(Theta_L)
  # print_theta() is a helper function that prints Theta_L and architecture info
  # It does not actually "do" anything except print to stdout

  T = length(Theta_L)

  println()
  println("NN ARCHITECTURE")
  println("$(T+1) Layers ($(T-1) Hidden)")
  println("$T Thetas")
  println("$(size(Theta_L[1],2)-1) Input Features")
  println("$(size(Theta_L[end], 1)) Output Classes")
  println()
    
  println("Units per layer (excl. bias unit)")
  for t = 1:T
    if t == 1
      println(" - Input: $(size(Theta_L[t],2)-1) Units")
    end
    if t < T
      println(" - Hidden $t: $(size(Theta_L[t],1)) Units")
    else
      println(" - Output: $(size(Theta_L[t],1)) Units")
    end
  end
  println()

  println("Theta Shapes")
  for l = 1:T
    println("Theta $l: $(size(Theta_L[l]))")
  end
  println()
  
  println("Theta Values")
  for t= 1:T
    println("Theta $t:" )
    println(Theta_L[t])
  end
  println()

end

In [ ]:
function nn_cost(Y, Y_pred)
  #
  # nn_cost implements cost function for array inputs Y and Y_pred
  #  
  # y is array of n_observations by n_classes
  # Y_pred is array with same dimensions as Y of predicted y values
  #
  if size(Y) != size(Y_pred)
    if size(Y,1) != size(Y_pred,1)
      error("Wrong number of predictions", "$(size(Y,1)) Actual Values. $(size(Y_pred,1)) Predicted Values. ")
    else
      error("Wrong number of prediction classes", "$(size(Y,2)) Actual Classes. $(size(Y_pred,2)) Predicted Classes. ")
    end
  end
    
  n_observations = size(Y,1)
  
  # Cost Function
  J = (-1.0 / n_observations ) * sum((Y .* log(Y_pred)) + ((1-Y) .* log(1-Y_pred)))
  
  J

end

In [ ]:
function nn_predict(X, Theta_L)
  #
  # nn_predict calculates activations for all layers given X and thetas in Theta_L
  # return all inputs and activations for all layers for use in backprop
  #
  # Parameters
  #  X is matrix of input features dimensions n_observations by n_features
  #  Theta_L is a 3D array where first element corresponds to the layer number, second is unit at layer+1, third is unit in layer
  #
  # Returns
  #  a_N - 1D Array of activation 2D arrays for each layer: Input (1), Hidden (2 to T), and Output (T+1)
  #  a_Z - 1D Array of input 2D arrays to compute activations for all non-bias units
  #  a_N[end] - 2D Array of predicted Y values with dimensions n_observations by n_classes
  #

  a_N = Any[]
  z_N = Any[]

  m = size(X,1)
  T = length(Theta_L)
    
  # Input Layer inputs
  push!(a_N, X) # List of activations including bias unit for non-output layers
  push!(z_N, zeros(1,1)) # add filler Z layer to align keys/indexes for a, z, and Theta

  # Loop through each Theta_List theta
  # t is index of Theta for calculating layer t+1 from layer t
  for t=1:T
    # Reshape 1D Array into 2D Array
    if ndims(a_N[t]) == 1
      a_N[t] = reshape(a_N[t], 1, size(a_N[t],1))
    end

    # Add bias unit
    a_N[t] = [ones(size(a_N[t],1), 1)  a_N[t]]
      
    # Calculate and Append new z and a arrays to z_N and a_N lists
    push!(z_N, a_N[t] * Theta_L[t]')
    push!(a_N, sigmoid(z_N[t+1]))
  end
  
  z_N, a_N, a_N[end]

end

In [ ]:
function back_prop(X_train, Y_train, Theta_L, lmda)
  #
  # Parameters
  #  X_train - Array of feature inputs with dimensions n_observations by n_features
  #  Y_train - Array of class outputs with dimensions n_observations by n_classes
  #  Theta_L is a 1D array of Theta values where 1D element is the layer number, the 2D elements are unit in layer+1 and unit in layer
  #  lmda - Float64 - lambda term for regularization
  #  
  # Returns
  #  Y_pred as array of predicted Y values from nn_predict()
  #  Theta_Gradient_L as 1D array of 2D Theta Gradient arrays
  #

  n_observations = size(X_train,1)
  
  T = length(Theta_L)

  # Create Modified copy of the Theta_L for Regularization with Coefficient for bias unit set to 0 so that bias unit is not regularized
  # Create variable to accumulate error caused by each Theta_L term in layer a_N[n+1]
  Theta_Gradient_L = Any[]
  regTheta = Any[]
  for i=1:T
    push!(regTheta, [zeros(size(Theta_L[i],1),1) Theta_L[i][:, :]])
    push!(Theta_Gradient_L, zeros(size(Theta_L[i])))
  end


  # Forward Pass
  z_N, a_N, Y_pred = nn_predict(X_train, Theta_L)

  # Backprop Error Accumulator
  delta_N = Any[]
  for t=1:T
    push!(delta_N, Any[])
  end
    
  # Error for Output layer is predicted value - Y training value
  delta = Y_pred - Y_train
  if ndims(delta) == 1
    delta = reshape(delta, 1, length(delta) )
  end

  # Loop backwards through Thetas to apply Error to prior Layer (except input layer)
  # Finish at T-2 because start at 0, output layer is done outside, the loop and input has no error

  # Output Error
  delta_N[T] = delta

  # Hidden Layers Error    
  for t=0:T-2
    delta = (delta * Theta_L[T-t][:, :]) .* sigmoidGradient(z_N[T-t])
    delta_N[T-t-1] = delta
  end
    
  # Calculate error gradients (no error in input layer)
  # t is the Theta from layer t to layer t+1
  for t=1:T
    Theta_Gradient_L[t] = Theta_Gradient_L[t] + delta_N[t]' * a_N[t] #'
  end

  # Average Error + regularization penalty  
  for t=1:T
    Theta_Gradient_L[t] = Theta_Gradient_L[t] * (1.0/n_observations) + (lmda * regTheta[t])
  end
  
  Y_pred, Theta_Gradient_L

end

In [ ]:
function fit(X_train, Y_train, Theta_L, lmda, epochs)
  #
  #fit() calls the training back_prop function for the given number of cycles
  #tracks error and error improvement rates
  #  
  #Parameters:
  #  X_train - Array of training data with dimension n_observations by n_features
  #  Y_train - Array of training classes with dimension n_observations by n_classes
  #  Theta_L - 1D array of theta 2d arrays where each theta has dimensions n_units[layer+1] by n_units[layer]+1
  #  epochs -  integer of number of times to update Theta_L
  #  
  #Returns
  #  Theta_L - 1D array of Theta arrays
  #  J_List - Array (length = epochs) of result of cost function for each iteration

  J_list = zeros(epochs)

  for i=1:epochs
    # Back prop to get Y_pred and Theta gradient
    Y_pred, Theta_grad = back_prop(X_train, Y_train, Theta_L, lmda)
    # Record cost
    J_list[i] = nn_cost(Y_train, Y_pred)
    # Update Theta using Learning Rate * Theta Gradient
    for t=1:length(Theta_L)
      # Start with a large learning rate; need to update this to be more intelligent than simply looking at iteration count
      # Need to update to change learning rate based on progress of cost function
      if i < 100
        learning_rate = 5.0        
      else
        learning_rate = 1.0
      end
      Theta_L[t] = Theta_L[t] - ( learning_rate * Theta_grad[t] )
    end
    #println("Cost $i: $(J_list[i])")
  end

  Theta_L, J_list

end

In [ ]:
# Fit
  Theta_L, J_list = fit(features, labels, Theta_L, lmda, epochs)

In [ ]:
# Hidden layer architecture
  hidden_unit_length_list = [2]
  hidden_layer_architecture = hidden_unit_length_list

In [ ]:
# Regularization Term
  lmda = 1e-5
  epochs = 100

In [ ]:
# Initialize Theta based on selected architecture
  Theta_L = initialize_theta(size(features,2), size(labels,2), 0)

In [ ]:
# Fit
  Theta_L, J_list = fit(features, labels, Theta_L, lmda, epochs)

In [ ]:
# Print Architecture
  print_theta(Theta_L)
  
# Print Cost
  println("Cost Function Applied to Training Data: $(J_list[end])")

# Predict
  X_new = [1 0;0 1;1 1;0 0]
  println( "Given X: $X_new")
  z_N, a_N, Y_pred = nn_predict(X_new, Theta_L)
  println( "Predicted Y: $(round(Y_pred,3))")

  Y_pred

In [ ]:
function XOR_test(hidden_unit_length_list, epochs)
  #
  #XOR_test is a simple test of the nn printing the predicted value to std out
  #Trains on a sample XOR data set
  #Predicts a single value
  #Accepts an option parameter to set architecture of hidden layers
  #
  
  
  println( "Training Data: X & Y")

  # Training Data
  X_train = [1 1; 1 0; 0 1; 0 0]	# Training Input Data
  Y_train = [0 1; 1 0; 1 0; 0 1] 			# Training Classes
  println( X_train )
  println( Y_train )
  
  # Hidden layer architecture
  hidden_layer_architecture = hidden_unit_length_list

  # Regularization Term
  lmda = 1e-5

  # Initialize Theta based on selected architecture
  Theta_L = initialize_theta(size(X_train,2), size(Y_train,2), hidden_layer_architecture)

  
  # Fit
  Theta_L, J_list = fit(X_train, Y_train, Theta_L, lmda, epochs)
  
  # Print Architecture
  print_theta(Theta_L)
  
  # Print Cost
  println("Cost Function Applied to Training Data: $(J_list[end])")

  # Predict
  X_new = [1 0;0 1;1 1;0 0]
  println( "Given X: $X_new")
  z_N, a_N, Y_pred = nn_predict(X_new, Theta_L)
  println( "Predicted Y: $(round(Y_pred,3))")

  Y_pred

end



Y_pred = XOR_test([2], 5000)